In [1]:
from dask.distributed import Client

client = Client("tls://localhost:8786")
client

<Client: 'tls://192.168.134.198:8786' processes=99 threads=198, memory=567.76 GiB>

### Loading full analysis fileset

In [2]:
from os import listdir

# TODO: replace with UL sample once we have it
with open('data/2017_preUL_nano/HHToBBVVToBBQQQQ_cHHH1.txt', 'r') as file:
    filelist = [f[:-1].replace('/eos/uscms/', 'root://xcache//') for f in file.readlines()]   # redirector must be replaced xcache at Nebraksa coffea-casa

fileset = {
    '2017_HHbbVV4q': filelist
}

for sample in listdir('data/2017_UL_nano/'):
    if 
with open('data/preUL', 'r') as file:
    filelist = [f[:-1].replace('redirector.t2.ucsd.edu', 'xcache') for f in file.readlines()]   # redirector must be replaced xcache at Nebraksa coffea-casa

fileset = {'2017': filelist}

['HWminusJ_HToWW_M125_13TeV_powheg_pythia8_TuneCP5.txt',
 'HZJ_HToWW_M125_13TeV_powheg_jhugen714_pythia8_TuneCP5.txt',
 'GravitonToHHToWWWW.txt',
 'GluGluToHHTo4V_node_cHHH1_TuneCP5_PSWeights_13TeV-powheg-pythia8.txt',
 'GluGluZH_HToWW_M125_13TeV_powheg_pythia8_TuneCP5.txt',
 'QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8.txt',
 'WJetsToLNu_HT-800To1200_TuneCP5_13TeV-madgraphMLM-pythia8.txt',
 'GravitonToHHToWWWW_part2.txt',
 'BulkGravitonToHHTo4Q_MX-600to6000_MH-15to250_part1_TuneCP5_13TeV-madgraph_pythia8.txt',
 'GluGluToHHTo2B2VLNu2J_node_cHHH1_TuneCP5_PSWeights_13TeV-powheg-pythia8.txt',
 'QCD_HT300to500_TuneCP5_13TeV-madgraph-pythia8.txt',
 'TTToHadronic_TuneCP5_13TeV-powheg-pythia8.txt',
 'GravitonToHHToWWWW_lnuqq.txt',
 'WJetsToLNu_HT-600To800_TuneCP5_13TeV-madgraphMLM-pythia8.txt',
 'GluGluHToWWToLNuQQ_M125_NNPDF31_TuneCP5_PSweights_13TeV_powheg_JHUGen710_pythia8.txt',
 'GravitonToHHToWWWW_part1.txt',
 'QCD_HT2000toInf_TuneCP5_13TeV-madgraph-pythia8.txt',
 'WJetsToLNu_HT-2500T

In [ ]:
from coffea import processor
from coffea.nanoevents import BaseSchema, NanoAODSchema
import time

import processors

import shutil
shutil.make_archive("processors", "zip", base_dir="processors")
client.upload_file("processors.zip");

tic = time.time()

exe_args = {
    "client": client,
    "savemetrics": True,
    "schema": NanoAODSchema,
    "align_clusters": True,
    "skipbadfiles": True
}

# load the sample filelist
with open('data/SingleMuon_2017.txt', 'r') as file:
    filelist = [f[:-1].replace('redirector.t2.ucsd.edu', 'xcache') for f in file.readlines()]   # redirector must be replaced xcache at Nebraksa coffea-casa

fileset = {'2017': filelist}

print("Waiting for at least one worker...")
client.wait_for_workers(1)

print("Found at least one worker...")

out, metrics = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=JetHTTriggerEfficienciesProcessor(ak15=True),   # replace with your processor
    executor=processor.dask_executor,
    executor_args=exe_args,  
#     maxchunks=10
)

elapsed = time.time() - tic

# can customize this
print(f"num: {out['num'].view(flow=True)}")
print(f"den: {out['den'].view(flow=True)}")

print(f"Metrics: {metrics}")
print(f"Finished in {elapsed:.1f}s")

In [17]:
out

{'den': Hist(
   Regular(50, 0, 1000, name='jet1pt', label='$p_T (GeV)$'),
   Regular(15, 0, 300, name='jet1msd', label='MassSD (GeV)'),
   storage=Double()) # Sum: 20166086.0 (33569280.0 with flow),
 'num': Hist(
   Regular(50, 0, 1000, name='jet1pt', label='$p_T (GeV)$'),
   Regular(15, 0, 300, name='jet1msd', label='MassSD (GeV)'),
   storage=Double()) # Sum: 902053.0 (1018816.0 with flow)}

In [13]:
del out['jet1pt']
del out['jet1msd']
del out['jet2pt']

In [18]:
import pickle
from os.path import exists
out_file = 'outPickles/out_ak15.pickle'   # make sure to change!!

if not exists(out_file):
    with open(out_file, 'wb') as filehandler:
        pickle.dump(out, filehandler)

## Saving trigger efficiencies

In [19]:
effs = out['num'] / out['den']

with open('corrections/trigEffs/AK15JetHTTriggerEfficiency_2017_2.hist', 'wb') as filehandler:
    pickle.dump(effs, filehandler)

In [14]:
# saving as a root THND - doesn't work for 3D histograms with uproot3... need pyroot?
import uproot3

ak15_trig_effs = uproot3.create('corrections/trigEffs/AK8JetHTTriggerEfficiency_2017.root')
ak15_trig_effs['efficiency_ptmass'] = effs.to_numpy(flow=True)

/opt/conda/lib/python3.8/site-packages/numpy/core/function_base.py:151: RuntimeWarning: invalid value encountered in multiply
  y *= step
/opt/conda/lib/python3.8/site-packages/numpy/core/function_base.py:161: RuntimeWarning: invalid value encountered in add
  y += start
